In [1]:
import pandas as pd
from tqdm import tqdm
import re
import os
import random
import nltk
import collections

random.seed(1991)

/home/prevot/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
candor_path = "/home/prevot/data/no_media/"
dirs = os.listdir(candor_path)
len(dirs)

cliffhanger = pd.read_csv(os.path.join(candor_path, dirs[0], "transcription/transcript_cliffhanger.csv"))


In [3]:
all_utts= []
for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    utts = cliffhanger['utterance'].tolist()
    utts = [re.sub(r"(\w)([\.\?\!])", r"\1 #", x).lower().split('#') for x in utts]
    flat_utts = [x for xs in utts for x in xs]
    flat_utts = [l.strip().replace(',',' #') for l in flat_utts if len(l.split()) > 3]
    all_utts = all_utts + flat_utts

In [31]:
inis = [u.split()[0] for u in all_utts]
collections.Counter(inis)

Counter({'i': 65746,
         'so': 47830,
         'yeah': 47712,
         'and': 31668,
         'um': 31638,
         'but': 25584,
         'oh': 18134,
         'like': 15996,
         "it's": 15975,
         'uh': 15057,
         'you': 14223,
         "i'm": 10750,
         'well': 10681,
         'it': 8929,
         'no': 6478,
         "that's": 6272,
         'what': 5138,
         'they': 5042,
         'we': 4557,
         'the': 4148,
         'mhm': 3743,
         'my': 3684,
         'okay': 3661,
         "i've": 3492,
         'that': 3162,
         'do': 3017,
         'how': 2891,
         'he': 2592,
         'yes': 2450,
         'is': 2440,
         'because': 2174,
         'just': 2101,
         "there's": 2100,
         'this': 2007,
         'she': 1968,
         'are': 1965,
         'have': 1939,
         'mm': 1859,
         'all': 1797,
         'right': 1574,
         'if': 1559,
         "they're": 1527,
         "he's": 1347,
         "you're": 1287,
 

In [5]:
# PREVIOUS LIST ['so','but',"because","well",'like']

DM_DISCOURSE = ['so','but','because','then'] #'and'
DM_ATTITUDE = ['oh','like','well']
DM_FEEDBACK = ['yeah','okay','mhm','yes','mm','right','maybe','good','nice']

ALL_DMS = DM_ATTITUDE + DM_DISCOURSE


In [6]:
cand_utts = []
for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    utts = cliffhanger['utterance'].tolist()
    utts = [re.sub(r"(\w)([^\w\s'])", r"\1 #", x).lower() for x in utts]
    utts = [x.split() for x in utts if 'uh huh' not in x.lower() and 'uh mhm' not in x.lower()]
    utts = [x for x in utts if len(x) > 4 and len(x) < 31]
    cand_utts = cand_utts + utts

inis = [u[0] for u in cand_utts if len(u)>4]
collections.Counter(inis)

Counter({'yeah': 56330,
         'i': 20537,
         'so': 16438,
         'mhm': 15608,
         'oh': 14952,
         'um': 9943,
         'okay': 9377,
         'and': 8455,
         'but': 8219,
         "it's": 5942,
         'uh': 5150,
         'you': 4996,
         'right': 4496,
         'like': 4393,
         "i'm": 4231,
         'well': 4029,
         'no': 4018,
         "that's": 3518,
         'it': 3238,
         'what': 3184,
         'yes': 3039,
         'mm': 2847,
         'all': 2565,
         'they': 1561,
         'how': 1447,
         'the': 1430,
         'we': 1412,
         'do': 1257,
         'that': 1219,
         'my': 1092,
         "i've": 999,
         'is': 987,
         'are': 860,
         'have': 825,
         'thank': 824,
         'just': 769,
         'this': 742,
         'he': 739,
         'hi': 665,
         'a': 658,
         "there's": 628,
         'good': 621,
         'not': 606,
         'she': 595,
         'nice': 587,
         "yo

In [10]:
SO_canditates = []
THEN_canditates = [] 
BECAUSE_canditates = [] 
BUT_canditates = []

for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    for item,row in cliffhanger.iterrows():
        utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", row['utterance']).lower().split('#')
        if len(utts)>1:
            pairs = nltk.bigrams(utts)
            for pair in pairs:
                if (30 > len(pair[0].split())>3) and (30 >len(pair[1].split())>3):
                    if pair[1].split()[0] == 'so':
                        SO_canditates.append(pair[0]+'<$>'+pair[1])
                    elif pair[1].split()[0] == 'then':
                        THEN_canditates.append(pair[0]+'<$>'+pair[1])
                    elif pair[1].split()[0] == 'because':
                        BECAUSE_canditates.append(pair[0]+'<$>'+pair[1])
                    elif pair[1].split()[0] == 'but':
                        BUT_canditates.append(pair[0]+'<$>'+pair[1])
                        

print(len(BECAUSE_canditates))
print(len(THEN_canditates))
CANDIDATES = [SO_canditates,THEN_canditates,BECAUSE_canditates,BUT_canditates]                       


all_candidates = []

for c in CANDIDATES:
    random.shuffle(c)
    all_candidates = all_candidates + c[0:125]

random.shuffle(all_candidates)
foutput = open('data_corpus/english_DM_SEM_mono.txt','w')
towrite_lines = '\n'.join(all_candidates)
foutput.writelines(towrite_lines)
foutput.close()

561
186


In [12]:
OH_canditates = []
LIKE_canditates = [] 
WELL_canditates = [] 

for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    for item,row in cliffhanger.iterrows():
        utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", row['utterance']).lower().split('#')
        if len(utts)>1:
            pairs = nltk.bigrams(utts)
            for pair in pairs:
                if (30 > len(pair[0].split())>3) and (30 >len(pair[1].split())>3):
                    if pair[1].split()[0] == 'oh':
                        OH_canditates.append(pair[0]+'<$>'+pair[1])
                    elif pair[1].split()[0] == 'like':
                        LIKE_canditates.append(pair[0]+'<$>'+pair[1])
                    elif pair[1].split()[0] == 'well':
                        WELL_canditates.append(pair[0]+'<$>'+pair[1])
                        

print(len(LIKE_canditates))
print(len(WELL_canditates))
print(len(OH_canditates))
CANDIDATES = [LIKE_canditates,WELL_canditates,OH_canditates]                       


all_candidates = []

for c in CANDIDATES:
    random.shuffle(c)
    all_candidates = all_candidates + c[0:167]

random.shuffle(all_candidates)
foutput = open('data_corpus/english_DM_ATT_mono.txt','w')
towrite_lines = '\n'.join(all_candidates)
foutput.writelines(towrite_lines)
foutput.close()

4988
584
1071


In [33]:
SO_canditates = []
THEN_canditates = [] 
BECAUSE_canditates = [] 
BUT_canditates = []

for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    cliffhanger['next_utt'] = cliffhanger['utterance'].shift(-1)
    cliffhanger['next_id'] = cliffhanger['turn_id'].shift(-1)
    for item,row in cliffhanger.iterrows():
        utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", row['utterance']).lower().split('#')[:-1]
        next_utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", str(row['next_utt'])).lower().split('#')[:-1]
        if utts !=[]:
            utt = utts[-1].strip()
        else: 
            utt = ''
        if next_utts !=[]:
            next_utt = next_utts[0].strip()
        else:
            next_utt = ''
        if (30 > len(utt.split()) > 3) and (30 > len(next_utt.split()) > 3) and (row['next_id']!=0):
            first_tok = next_utt.split()[0]
            if first_tok.lower() == 'so':
                SO_canditates.append(utt.lower()+'<$>'+next_utt.lower())
            elif first_tok.lower() == 'then':
                THEN_canditates.append(utt.lower()+'<$>'+next_utt.lower())
            elif first_tok.lower() == 'because':
                BECAUSE_canditates.append(utt.lower()+'<$>'+next_utt.lower())
            elif first_tok.lower() == 'but':
                BUT_canditates.append(utt.lower()+'<$>'+next_utt.lower())


print(len(SO_canditates))
print(len(BUT_canditates))               
print(len(BECAUSE_canditates))
print(len(THEN_canditates))
CANDIDATES = [SO_canditates,BECAUSE_canditates,BUT_canditates]                       


all_candidates = []

for c in CANDIDATES:
    random.shuffle(c)
    all_candidates = all_candidates + c[0:158]

random.shuffle(all_candidates)
foutput = open('data_corpus/english_DM_SEM_dial.txt','w')
towrite_lines = '\n'.join(all_candidates)
foutput.writelines(towrite_lines)
foutput.close()

4379
2163
158
35


In [ ]:
OH_canditates = []
LIKE_canditates = [] 
WELL_canditates = [] 
cnt = 0
cnt2 = 0
cnt3 = 0

for i in dirs:
    cnt+=1
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    cliffhanger['next_utt'] = cliffhanger['utterance'].shift(-1)
    cliffhanger['next_id'] = cliffhanger['turn_id'].shift(-1)
    for item,row in cliffhanger.iterrows():
        cnt2 +=1 
        utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", row['utterance']).lower().split('#')[:-1]
        next_utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", str(row['next_utt'])).lower().split('#')[:-1]
        if utts !=[]:
            utt = utts[-1].strip()
        else: 
            utt = ''
        if next_utts !=[]:
            next_utt = next_utts[0].strip()
        else:
            next_utt = ''
        if (30 > len(utt.split()) > 3) and (30 > len(next_utt.split()) > 3) and (row['next_id']!=0):
            first_tok = next_utt.split()[0]
            cnt3+=1
            if first_tok.lower() == 'oh':
                OH_canditates.append(utt.lower()+'<$>'+next_utt.lower())
            elif first_tok.lower() == 'like':
                LIKE_canditates.append(utt.lower()+'<$>'+next_utt.lower())
            elif first_tok.lower() == 'well':
                WELL_canditates.append(utt.lower()+'<$>'+next_utt.lower())


                
print(cnt)
print(cnt2)
print(cnt3)

print(len(LIKE_canditates))
print(len(WELL_canditates))
print(len(OH_canditates))
CANDIDATES = [LIKE_canditates,WELL_canditates,OH_canditates]                       


all_candidates = []

for c in CANDIDATES:
    random.shuffle(c)
    all_candidates = all_candidates + c[0:167]

random.shuffle(all_candidates)
foutput = open('data_corpus/english_DM_ATT_dial.txt','w')
towrite_lines = '\n'.join(all_candidates)
foutput.writelines(towrite_lines)
foutput.close()


In [ ]:
mono_canditates = []
cnt = 0
cnt2 = 0
cnt3 = 0

for i in dirs:
    cnt+=1
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    for item,row in cliffhanger.iterrows():
        cnt2 +=1 
        utts = re.sub(r"(\w)([\.\?\!])", r"\1 #", row['utterance']).lower().split('#')
        if len(utts)>1:
            pairs = nltk.bigrams(utts)
            for pair in pairs:
                cnt3 +=1
                if (30 > len(pair[0].split())>3) and (30 >len(pair[1].split())>3):
                    if pair[1].split()[0] in DM_ATTITUDE:
                        mono_canditates.append(pair[0]+'<$>'+pair[1])

print(cnt)
print(cnt2)
print(cnt3)
print(len(mono_canditates))
random.shuffle(mono_canditates)
mono_canditates = mono_canditates[0:500]
foutput = open('data_corpus/english_DM_ATT_mono.txt','w')
towrite_lines = '\n'.join(mono_canditates)
foutput.writelines(towrite_lines)
foutput.close()

In [52]:
DMS = ['so','but',"because","well",'like']
mono_canditates = []
cnt = 0
cnt2 = 0
cnt3 = 0

for i in dirs:
    cnt+=1
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    for item,row in cliffhanger.iterrows():
        cnt2 +=1 
        utts = row['utterance'].lower().split('.')
        #print(utts)
        #input('???')
        if len(utts)>1:
            pairs = nltk.bigrams(utts)
            for pair in pairs:
                cnt3 +=1
                if (20 > len(pair[0].split())>3) and (20 >len(pair[1].split())>3):
                    if pair[1].split()[0] in DMS:
                        mono_canditates.append(pair[0]+'<$>'+pair[1])

print(cnt)
print(cnt2)
print(cnt3)
print(len(mono_canditates))

1656
530316
974368
14054


In [53]:
random.shuffle(mono_canditates)
mono_canditates = mono_canditates[0:500]
foutput = open('data_corpus/english_DM_mono.txt','w')
towrite_lines = '\n'.join(mono_canditates)
foutput.writelines(towrite_lines)
foutput.close()

In [50]:
DMS = ['so','but',"because","well",'like']
dial_canditates = []
dial_dm_stats = []

for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    cliffhanger['next_utt'] = cliffhanger['utterance'].shift(-1)
    cliffhanger['next_id'] = cliffhanger['turn_id'].shift(-1)
    
    for i,row in cliffhanger.iterrows():
        if (20 > len(row['utterance'].split()) > 3) and (20 > len(str(row['next_utt']).split()) > 3) and (row['next_id']!=0):
            first_tok = row['next_utt'].split()[0]
            if first_tok.lower() in DMS: 
                dial_canditates.append(row['utterance'].lower()+'<$>'+row['next_utt'].lower())
                dial_dm_stats.append(first_tok)

print(collections.Counter(dial_dm_stats))
print(len(dial_canditates))

Counter({'So': 4133, 'But': 1852, 'Like': 896, 'Well': 856, 'Because': 97, 'but': 5, 'so': 5, 'well': 2, 'like': 1})
7847


In [51]:
random.shuffle(dial_canditates)
dial_canditates = dial_canditates[0:500]
foutput = open('data_corpus/english_DM_dial.txt','w')
towrite_lines = '\n'.join(dial_canditates)
foutput.writelines(towrite_lines)
foutput.close()         
                    
